In [1971]:
# LIBRARIES
import pandas as pd
import numpy as np
import json
from fuzzywuzzy import process
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

In [1923]:
def read_data(file_name, header=0, sep=','):
    """
    Reads a csv file and returns a pandas dataframe
    """
    return pd.read_csv(file_name, sep=sep, header=header)

In [1924]:
def try_parse_float(value):
    """
    Try to parse a string as a float
    """
    try:
        float(value)
    except:
        return False
    return True

In [1925]:
def detect_correct_datatype(df, column):
    float_count = df[column].apply(lambda x: try_parse_float(x)).sum() - df[column].isna().sum()
    percentage_float = ((df.shape[0] - float_count) / df.shape[0]) * 100
    if percentage_float <= 25:
        df[column] = pd.to_numeric(df[column], errors='coerce')

In [1926]:
def convert_type(val, type):
    """
    Convert a value to a given type
    """
    try:
        if type == 'int64':
            return np.int64(val)
        elif type == 'float64':
            return np.float64(val)
        elif type == 'uint8':
            return np.uint8(val)
    except:
        return None

In [1927]:
def is_numeric_or_categorical(df, column):
    unique_values = df[column].unique()
    if ((df.shape[0] - len(unique_values)) / df.shape[0]) * 100 < 93:
        return True, ((df.shape[0] - len(unique_values)) / df.shape[0]) * 100
    return False, ((df.shape[0] - len(unique_values)) / df.shape[0]) * 100

In [1928]:
def remove_columns(df, columns):
    """
    Removes columns from a dataframe
    """
    df.drop(columns, axis=1, inplace=True)

In [1929]:
def remove_rows(df, rows):
    """
    Removes rows from a dataframe
    """
    df.drop(rows, axis=0, inplace=True)

In [1930]:
def drop_rows_condition(df, conition):
    return df[~conition]

In [1931]:
def remove_duplicates(df):
    """
    Removes duplicates from a dataframe
    """
    df.drop_duplicates(inplace=True)

In [1932]:
def convert_to_datatype(df, column, datatype):
    """
    Converts a column to a datatype
    """
    if datatype == 'object':
        df[column] = df[column].astype(datatype)
    else:
        df[column] = df[column].apply(lambda x: convert_type(x, datatype))
        
    return df

In [1933]:
def remove_outlier_numeric(df, column, min=-np.inf, max=np.inf):
    """
    Removes rows from a dataframe based on a condition
    """
    if min != -np.inf and max != np.inf:
        return df[((df[column] >= min) & (df[column] <= max)) | df[column].isna()]
    elif min == -np.inf and max != np.inf:
        return df[(df[column] <= max) | df[column].isna()]
    elif min != -np.inf and max == np.inf:
        return df[(df[column] >= min) | df[column].isna()]

In [1934]:
def remove_outlier_categorical(df, column, unqiue_values):
    """
    Removes rows from a dataframe based on a condition
    """
    inconsistent_categories = pd.array(list(set(df[column].unique()) - set(unqiue_values)))
    return df[(~df[column].isin(inconsistent_categories)) | df[column].isna()]

In [1935]:
def detect_outliers_std(df, column, std=3):
    """
    Detect outliers based on standard deviation
    """
    
    # Set upper and lower limit to 3 standard deviation
    random_data_std = np.std(df[column])
    random_data_mean = np.mean(df[column])
    anomaly_cut_off = random_data_std * std
    
    lower_limit  = random_data_mean - anomaly_cut_off 
    upper_limit = random_data_mean + anomaly_cut_off

    # Generate outliers
    outliers = (df[column] < lower_limit) | (df[column] > upper_limit)
    return outliers

In [1936]:
def detect_outliers_isolation_forest(df, column, contamination=0.1):
    """
    Detect outliers based on isolation forest
    """
    # Create isolation forest
    clf = IsolationForest(random_state=0, contamination=contamination)
    predictions = clf.fit_predict(df[column].to_numpy().reshape(-1, 1))   
    return predictions == -1

In [1937]:
def correct_category_levenshtein(df, column, incorrect_categories, threshold=80):
    """
    Corrects a column by using fuzzywuzzy to find the correct category
    """    
    # inconsistent_categories = pd.array(list(set(df[column].unique()) - set(incorrect_categories)))
    inconsistent_categories = pd.array(list(incorrect_categories))
    inconsistent_categories = inconsistent_categories[~inconsistent_categories.isna()]
    
    correct_categories = pd.array(list(set(df[column].unique()) - set(incorrect_categories)))
    
    for inconsistent_category in inconsistent_categories:
        if not pd.isna(inconsistent_category):
            potential_match = process.extractOne(inconsistent_category, correct_categories)
            if potential_match[1] > threshold:
                df.loc[df[column] == inconsistent_category, column] = potential_match[0]
    return df

In [1938]:
def convert_nominal_categories(df, columns):
    """
    Converts categorical data to numeric data
    """
    return pd.get_dummies(df, columns=columns)

In [1939]:
def convert_ordinal_category(df, column, order):
    df[column].replace(to_replace=df[column].unique(), value=order, inplace=True)

In [1940]:
def fill_missing_values(df, column, value):
    """
    Fills missing values in a column with a value
    """
    df[column].fillna(value, inplace=True)

In [1941]:
def drop_missing_values(df, column):
    """
    Drops missing values in a column
    """
    df.dropna(subset=[column], inplace=True)

In [1942]:
def fill_average_mode(df, column, is_numeric):
    """
    Fills missing values in a column with the average or mode
    """
    if not is_numeric:
        df[column].fillna(df[column].mode()[0], inplace=True)
    else:
        df[column].fillna(df[column].mean(), inplace=True)

In [1943]:
def knn_impute(df, column, is_numeric):
    """
    Imputation using KNN
    """
    x_train = df[~df[column].isna()].copy()
    x_train.dropna(inplace=True)
    
    y_train = x_train[column]
    x_train = x_train[x_train.columns[x_train.columns != column]]
    x_train = x_train[x_train.columns[x_train.dtypes != 'object']]
     
        
    x_predict = df[df[column].isna()][df.columns[df.columns != column]].copy()
    x_predict = x_predict[x_predict.columns[x_predict.dtypes != 'object']]
    
    if x_predict.shape[0] == 0:
        return
    
    if is_numeric:
        # REGRESSION
        knn_regressor = KNeighborsRegressor()
        knn_regressor.fit(x_train, y_train)
        y_predict = knn_regressor.predict(x_predict)
        df.loc[df[column].isna(), column] = y_predict
    else:
        # CLASSIFICATION
        knn_classifier = KNeighborsClassifier()
        knn_classifier.fit(x_train, y_train)
        y_predict = knn_classifier.predict(x_predict)
        df.loc[df[column].isna(), column] = y_predict

In [1944]:
def automatic_data_filler(df, column, output_column, is_numeric, no_corr=0.01, low_corr=0.5):
    if df[column].isna().sum() == 0:
        print(column, ': No missing values')
        return 'automatic'
    
    df_temp = df.copy()
    drop_missing_values(df_temp, column)
    if df_temp[column].dtype == 'object':
        convert_ordinal_category(df_temp, column, [x for x in range(len(df_temp[column].unique()))])
        
    p_score = df_temp[column].corr(df_temp[output_column], method='pearson')
    
    if p_score >= -no_corr and p_score <= no_corr:
        # NO CORRELATION
        if ((df.shape[0] - df[column].isna().sum()) / df.shape[0]) * 100 >= 50:
            # MISSING VALUES ARE TOO LARGE
            # print(column, p_score, "No correlation, missing values too large")
            remove_columns(df, [column])
            return 'column'
        else:
            # MISSING VALUES ARE SMALL
            # print(column, p_score, "No correlation, missing values small")
            drop_missing_values(df, column)
            return 'row'
    elif (p_score >= -low_corr and p_score < -no_corr) or (p_score > no_corr and p_score <= low_corr):
        # LOW CORRELATION
        # print(column, p_score, "Low correlation")
        fill_average_mode(df, column, is_numeric)
        return 'average'
    elif p_score >= -1 and p_score <= 1:
        # HIGH CORRELATION
        # print(column, p_score, "High correlation")
        knn_impute(df, column, is_numeric)
        return 'knn'

In [1963]:
def data_cleaning_suggestions(df, output_column):
    df_jsons = []
    for col in df.columns[df.columns != output_column]:
        col_json = {}
        col_json.update({"column_name": col})
        
        # Detect Datatype
        detect_correct_datatype(df, col)
        if df[col].dtype == 'object':
            col_json.update({"datatype": "object"})
        elif df[col].dtype == 'int64':
            col_json.update({"datatype": "int64"})
        elif df[col].dtype == 'float64':
            col_json.update({"datatype": "float64"})
        elif df[col].dtype == 'uint8':
            col_json.update({'datatype': 'uint8'})
        
        # Detect Numeric or Categorical
        is_numeric = False
        if df[col].dtype != 'object':
            is_numeric, _ = is_numeric_or_categorical(df, col)
        col_json.update({'is_numeric': is_numeric})
        
        # Detect Outliers
        if is_numeric:
            outliers = detect_outliers_std(df, col)
            df = drop_rows_condition(df, outliers)
        elif df[col].dtype != 'object':
            outliers = detect_outliers_std(df, col)
            df = drop_rows_condition(df, outliers)
        elif df[col].dtype == 'object':
            df[col] = df[col].str.strip()
            df_temp = df.copy()
            convert_ordinal_category(df_temp, col, [x for x in range(len(df_temp[col].unique()))])
            outliers = detect_outliers_isolation_forest(df_temp, col, 0.05)
            try:
                df = correct_category_levenshtein(df, col, df[outliers][col].unique())
            except:
                print(col, "ERROR: Levenshtein")
                df = drop_rows_condition(df, outliers)
                        
        if is_numeric:
            col_json.update({'min': df[col].min(), 'max': df[col].max(), 'mean': df[col].mean()})
            col_json.update({'unique_count': None, 'unique_values': None})
        elif df[col].dtype != 'object':
            col_json.update({'min': df[col].min(), 'max': df[col].max(), 'mean': df[col].mean()})
            col_json.update({'unique_count': len(df[col].unique()), 'unique_values': df[col].unique().tolist()})
        elif df[col].dtype == 'object':
            col_json.update({'min': None, 'max': None, 'mean': None})
            col_json.update({'unique_count': len(df[col].unique()), 'unique_values': df[col].unique().tolist()})
            
        # Missing Data Filler
        method = automatic_data_filler(df, col, output_column, is_numeric)
        col_json.update({'fill_method': method})
        
        if method != 'column' and not is_numeric:
            col_json.update({'unique_count': len(df[col].unique()), 'unique_values': df[col].unique().tolist()})
        
        # completing json
        col_json.update({'is_nominal': True})
        col_json.update({'ordinal_order': []})
                
        df_jsons.append(col_json)
            
    return df_jsons, df

In [1965]:
def clean_data(df, output_column, operations):
    for col_json in operations:
        # Column name
        col = col_json['column_name']
        
        # Check if column dropped
        if col_json['fill_method'] == 'column':
            remove_columns(df, [col])
            continue
        
        # Convert to datatype
        convert_to_datatype(df, col, col_json['datatype'])
                
        # Categorical or numeric
        is_numeric = col_json['is_numeric']
        print(col, df.empty, df.shape, "OUTLIER")
        # Detect Outliers
        if is_numeric:
            df = remove_outlier_numeric(df, col, col_json['min'], col_json['max'])
        elif df[col].dtype != 'object':
            df = remove_outlier_categorical(df, col, col_json['unique_values'])
        elif df[col].dtype == 'object':
            df[col] = df[col].str.strip()
            outliers = pd.array(list(set(df[col].unique()) - set(col_json['unique_values'])))
            try:
                df = correct_category_levenshtein(df, col, outliers.to_numpy())
            except:
                df = drop_rows_condition(df, df[col].isin(outliers))
        print(col, df.empty, df.shape)
        # Fill missing data
        if col_json['fill_method'] == 'automatic':
            _ = automatic_data_filler(df, col, output_column, is_numeric)
        elif col_json['fill_method'] == 'average':
            fill_average_mode(df, col, is_numeric)
        elif col_json['fill_method'] == 'knn':
            knn_impute(df, col, is_numeric)
        elif col_json['fill_method'] == 'row':
            drop_missing_values(df, col)
                        
        # Convert Nominal/Ordinal
        if df[col].dtype == 'object':
            if col_json['is_nominal']:
                df = convert_nominal_categories(df, [col])
            else:
                convert_ordinal_category(df, col, col_json['ordinal_order'])
        print(col, df.empty, df.shape)
    return df

# TESTING FUNCTIONS

In [1976]:
titanic_df = read_data('../data/raw/titanic/titanic.csv', header=0)
# remove_columns(titanic_df, ['PassengerId', 'Pclass', 'Ticket', 'Cabin', 'Age', 'Sex', 'SibSp', 'Parch', 'Fare', 'Name'])
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [1977]:
df = titanic_df.copy()
test_json, df = data_cleaning_suggestions(df, 'Survived')
with open('suggestions.json', "w") as outfile:
        outfile.write(json.dumps(test_json))

PassengerId : No missing values
Pclass : No missing values
Name : No missing values
Sex : No missing values
SibSp : No missing values
Parch : No missing values
Ticket : No missing values
Fare : No missing values


In [1682]:
remove_columns(df, ['PassengerId', 'Name', 'Ticket', 'Cabin'])
convert_ordinal_category(df, 'Sex', [0, 1])
df = convert_nominal_categories(df, ['Embarked'])
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,0,0,1
1,1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,1


In [1978]:
operations = json.load(open('suggestions.json'))
test_df = titanic_df.copy()
test_df = clean_data(test_df, 'Survived', operations)
test_df.head()

Pclass False (891, 11) OUTLIER
Pclass False (891, 11)
Pclass : No missing values
Pclass False (891, 11)
Sex False (891, 10) OUTLIER
Sex False (891, 10)
Sex : No missing values
Sex False (891, 10)
Age False (891, 10) OUTLIER
Age False (889, 10)
Age False (889, 10)
SibSp False (889, 10) OUTLIER
SibSp False (859, 10)
SibSp : No missing values
SibSp False (859, 10)
Parch False (859, 10) OUTLIER
Parch False (844, 10)
Parch : No missing values
Parch False (844, 10)
Fare False (844, 9) OUTLIER
Fare False (825, 9)
Fare : No missing values
Fare False (825, 9)
Embarked False (825, 8) OUTLIER
Embarked False (825, 8)
Embarked False (825, 10)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,0,0,1
1,1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,1


In [1980]:
print(df.shape)
print(test_df.shape)
print(df.columns)

(825, 12)
(825, 10)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [1686]:
df.compare(test_df)

Empty DataFrame
Columns: []
Index: []

In [1687]:
# df.sort_values(by=['Age'], inplace=True)
# test_df.sort_values(by=['Age'], inplace=True)
df.to_csv('suggestions.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [1981]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_log_error

In [1983]:

x_train, x_test, y_train, y_test = train_test_split(test_df.loc[:, test_df.columns != 'Survived'], test_df['Survived'], test_size=0.2)

clf = SVC(kernel='linear', random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy_score(y_test, y_pred)

0.8181818181818182

In [1984]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       104
           1       0.78      0.70      0.74        61

    accuracy                           0.82       165
   macro avg       0.81      0.79      0.80       165
weighted avg       0.82      0.82      0.82       165



In [1966]:
cars = read_data('../data/raw/cars/cars.csv', header=0)
cars.head()
cars['model'].unique()
# cars['model'] = cars['model'].str.strip()
# cars['model'].unique()

array([' A1', ' A3', ' Q2', ' Q3', ' A5', ' Q5', ' A4', ' A6', nan,
       ' A1.', ' A5.'], dtype=object)

In [1967]:
df = cars.copy()
test_json, df = data_cleaning_suggestions(cars, 'price')
with open('suggestions.json', "w") as outfile:
        outfile.write(json.dumps(test_json))
cars['year'].value_counts()     

ID : No missing values
year : No missing values
transmission : No missing values
fuelType : No missing values
tax : No missing values
mpg : No missing values
engineSize : No missing values
ownerName : No missing values


2019    2242
2016    1417
2017    1403
2015     697
2018     574
2020     449
2014     300
2013     207
2012      57
2011      29
2010      20
2009      15
2008      13
2007       6
2006       4
2005       4
2004       1
Name: year, dtype: int64

In [1968]:
operations = json.load(open('suggestions.json'))
test_df = cars.copy()
test_df = clean_data(test_df, 'price', operations)
test_df.head()

model False (7438, 11) OUTLIER
model False (7438, 11)
model False (7438, 11)
year False (7438, 11) OUTLIER
year False (7375, 11)
year : No missing values
year False (7375, 11)
transmission False (7375, 11) OUTLIER
transmission False (7375, 11)
transmission : No missing values
transmission False 

C:\Users\omar_\AppData\Local\Temp\ipykernel_8836\1929179420.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(datatype)
C:\Users\omar_\AppData\Local\Temp\ipykernel_8836\2367574136.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip()


(7375, 13)
mileage False (7375, 13) OUTLIER
mileage False (7277, 13)
mileage False (7277, 13)
fuelType False (7277, 13) OUTLIER
fuelType False (7277, 13)
fuelType : No missing values
fuelType False (7277, 15)
tax False (7277, 15) OUTLIER
tax False (7269, 15)
tax : No missing values
tax False (7269, 15)
mpg False (7269, 15) OUTLIER
mpg False (7233, 15)
mpg : No missing values
mpg False (7233, 15)
engineSize False (7233, 15) OUTLIER
engineSize False (7233, 15)
engineSize : No missing values
engineSize False (7233, 15)
state False (7233, 15) OUTLIER
state False (7233, 15)
state False (7233, 16)


,model,year,price,mileage,tax,mpg,engineSize,transmission_Automatic,transmission_Manual,transmission_Semi-Auto,fuelType_Diesel,fuelType_Hybrid,fuelType_Petrol,state_New,state_Used
0,0,2016,10999,19865.0,0,76.3,3,0,1,0,1,0,0,0,1
1,1,2017,18400,27672.0,125,49.6,5,0,1,0,0,0,1,0,1
2,1,2016,15945,27306.0,125,61.4,5,0,0,1,1,0,0,0,1
3,1,2015,8490,40000.0,20,68.9,5,0,1,0,1,0,0,0,1
4,5,2018,17297,31433.0,145,64.2,3,0,1,0,1,0,0,0,1


In [1969]:
test_df.shape

(7233, 15)

In [1972]:
x_train, x_test, y_train, y_test = train_test_split(
    test_df.loc[:, test_df.columns != 'price'], test_df["price"], test_size=0.20)

In [1973]:
rf = RandomForestRegressor(n_estimators=1000, random_state=0)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print("RF error: ", mean_squared_log_error(y_test, y_pred, squared=True))
print("RF error: ", mean_squared_log_error(y_test, y_pred, squared=False))

RF error:  0.008168815686525662
RF error:  0.09038150079814819
